In [1]:
import re
from math import ceil, cos, sin, tan, pi, radians, sqrt, degrees
from renard import R40, find_greater_than_or_equal

In [2]:
spec = 'EXT FLAT ROOT DIA FIT 12/24 18 30 ANSI B92'
# spec = 'INT FLAT ROOT SIDE FIT 16/32 11 30 BS 3550'
# spec = 'EXT 25z x 1,0m x 30R x 6e - ISO 4156'

In [3]:
spec_list = [
    i for i in re.split(r'x| |/|-|ISO|4156|ANSI|B92|ROOT|FIT|BS|3550', spec)
    if bool(i)
]
if 'ANSI' in spec:
    spline_type, spline_root, spline_fit, diametral_pitch, stub_pitch, teeth, pressure_angle = spec_list
    diametral_pitch = float(diametral_pitch)
    pressure_angle = float(pressure_angle)
    teeth = int(teeth)
    stub_pitch = float(stub_pitch)
elif 'BS' in spec:
    spline_type, spline_root, spline_fit, diametral_pitch, stub_pitch, teeth, pressure_angle = spec_list
    diametral_pitch = float(diametral_pitch)
    pressure_angle = float(pressure_angle)
    teeth = int(teeth)
    stub_pitch = float(stub_pitch)
elif 'ISO' in spec:
    spline_type, teeth, module, pressure_angle, tolerance = spec_list
    teeth = int(teeth[:-1])
    if pressure_angle[-1] == 'R': root = 'fillet'
    if pressure_angle[-1] == 'P': root = 'flat'
    pressure_angle = float(pressure_angle[:-1])
    try:
        module = float(module[:-1])
    except ValueError:
        module = float(module[:-1].replace(',', '.'))
    tolerance_class = int(tolerance[0])
    fit_class = tolerance[1:]

In [5]:
pitch_dia = teeth / diametral_pitch
base_dia = pitch_dia * cos(radians(pressure_angle))
circular_pitch = pi / float(diametral_pitch)

# see Table 107 ANSI B92.1-1996
DIA_TOLERANCES = {
    range(1, 3): {
        'TAB': 0.0200,
        'FN': lambda P: round((2000 / P + 250) * 1e-4, ndigits=4)
    },
    range(3, 4): {
        'TAB': 0.0150,
        'FN': lambda P: round((2000 / P + 200) * 1e-4, ndigits=4)
    },
    range(4, 5): {
        'TAB': 0.0100,
        'FN': lambda P: round((2000 / P + 150) * 1e-4, ndigits=4)
    },
    range(5, 6): {
        'TAB': 0.0080,
        'FN': lambda P: round((2000 / P + 130) * 1e-4, ndigits=4)
    },
    range(6, 32): {
        'TAB': 0.0050,
        'FN': lambda P: round((2000 / P + 100) * 1e-4, ndigits=4)
    },
    range(32, 64): {
        'TAB': 0.0030,
        'FN': lambda P: round((2000 / P + 80) * 1e-4, ndigits=4)
    },
    range(64, 160): {
        'TAB': 0.0020,
        'FN': lambda P: round((2000 / P + 70) * 1e-4, ndigits=4)
    },
}

for pitch_range in DIA_TOLERANCES:
    if diametral_pitch in pitch_range:
        dia_tolerance = DIA_TOLERANCES[pitch_range]

rad_form_clearance = min(max(0.001 * pitch_dia, 0.002), 0.01)

form_dia_dict = {
    'EXT': {
        30.0: lambda N, P: (N - 1) / P - 2 * rad_form_clearance,
        37.5: lambda N, P: (N - 0.8) / P - 2 * rad_form_clearance,
        45.0: lambda N, P: (N - 0.6) / P - 2 * rad_form_clearance,
    },
    'INT': {
        'SIDE': lambda N, P: (N + 1) / P + 2 * rad_form_clearance,
        'DIA': lambda N, P: (N + 0.8) / P + 2 * rad_form_clearance - 0.004,
    },
}

# major_dia_dict = {
#     'EXT': lambda N, P: (N - 0.6) / P - 2 * rad_form_clearance,
#     'INT': {
#         'SIDE': lambda N, P: (N + 1) / P + 2 * rad_form_clearance,
#         'DIA': lambda N, P: (N + 0.8) / P + 2 * rad_form_clearance - 0.004,
#     },
# }

if spline_type == 'EXT':
    form_dia = form_dia_dict[spline_type][pressure_angle](teeth,
                                                          diametral_pitch)
    min_form_ext_dia = sqrt(
        3 * teeth**2 + (teeth - 0.016 * diametral_pitch - 4.5)**2) / (
            2 * diametral_pitch) if spline_root == 'FLAT' else sqrt(
                3 * teeth**2 + (teeth - 5.359)**2) / (2 * diametral_pitch)
    if spline_fit == 'SIDE':
        max_major_ext_dia = (teeth + 1) / diametral_pitch
        if spline_root == 'FLAT':
            min_major_ext_dia = max_major_ext_dia - dia_tolerance['FN'](
                diametral_pitch)
            form_ext_dia = max(min_form_ext_dia, form_dia)
        elif spline_root == 'FILLET':
            min_major_ext_dia = max_major_ext_dia - dia_tolerance['TAB']
            form_ext_dia = max(min_form_ext_dia, form_dia)
    elif spline_fit == 'DIA':
        form_ext_dia = max(min_form_ext_dia, form_dia)
        max_major_ext_dia = (teeth + 1) / diametral_pitch - 0.0001
        min_major_ext_dia = max_major_ext_dia - round(
            (3 + 2 * pitch_dia) * 1e-4, ndigits=4)
elif spline_type == 'INT':
    form_dia = form_dia_dict[spline_type][spline_fit](teeth, diametral_pitch)
    if pressure_angle == 30:
        min_eff_width = pi / (2 * diametral_pitch)
        min_minor_int_dia = (teeth - 1) / diametral_pitch
        if spline_root == 'FLAT':
            if spline_fit == 'SIDE':
                max_major_int_dia = (teeth + 1.35) / diametral_pitch
            elif spline_fit == 'DIA':
                max_major_int_dia = (teeth + 1) / diametral_pitch
        elif spline_root == 'FILLET':
            max_major_int_dia = (teeth + 1.8) / diametral_pitch
    elif pressure_angle == 37.5:
        min_eff_width = (0.5 * pi + .1) / (diametral_pitch)
        min_minor_int_dia = (teeth - 0.8) / diametral_pitch
        max_major_int_dia = (teeth + 1.6) / diametral_pitch
    elif pressure_angle == 45:
        min_eff_width = (0.5 * pi + .2) / (diametral_pitch)
        min_minor_int_dia = (teeth - 0.6) / diametral_pitch
        max_major_int_dia = (teeth + 1.4) / diametral_pitch